In [1]:
!pip install reload

row_sample_rate = 0.1

In [2]:
import numpy as np
import pandas as pd
import time
import reload

from helpers import clone_drop_and_convert, clone_impute_data, perf_measures, round_decision, reduce_dataframe_using_min_non_null, get_coefficent_weights
from models import apply_linear_regression_model
from predict_country import predict_country

# Developer note: If you modify the modules above, the changes to those modules will not be reflected until you 
# re-execute this cell, specifically the reload commands
# TODO this doesn't work yet
# reload(clone_drop_and_convert)
# reload(models)

DUMMY_NA = True
FILL="Mean"

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 255

script_start = time.time()

raw_df = pd.read_csv('./assets/survey_results_public.csv')
schema = pd.read_csv('./assets/survey_results_schema.csv')

In [3]:
df = clone_drop_and_convert(raw_df)

full_country = df.sample(frac=row_sample_rate, axis=0)
full_country.shape
# (6446, 57)

start = time.time()
full_country_imputed = clone_impute_data(full_country, dummy_na=True, fill_technique=FILL)
end = time.time()
print('full_country impute time %s', end - start)
print("full country dataset has %s rows and %s columns" % (full_country_imputed.shape[0], full_country_imputed.shape[1]))


full_country impute time %s 5.375078916549683
full country dataset has 6446 rows and 19045 columns


In [4]:
countries = full_country['Country'].value_counts()
countries_with_enough_samples = countries[countries > (1000 * row_sample_rate)].index.values
countries_with_enough_samples

array(['United States', 'India', 'Germany', 'United Kingdom', 'Brazil',
       'Canada', 'France', 'Netherlands', 'Poland', 'Australia', 'Italy'],
      dtype=object)

In [5]:
#cutoffs here pertains to the number of missing values allowed in the used columns.
#Therefore, lower values for the cutoff provides more predictors in the model.

# cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 25, 0]
cutoffs = [10000, 5000, 2500, 1000, 500]

results = []
for country in countries_with_enough_samples:
    country_results = predict_country( \
        rowset_label='full_country_%s' % country, \
        columnset_label='all_numeric_categerical_no_multichoice', \
        imputed_df=full_country_imputed, \
        cutoffs=cutoffs, \
        country_to_classify=country, \
        row_sample_rate=row_sample_rate \
    )
    results.extend(country_results)

rowset_label=full_country_United States, columnset_label=all_numeric_categerical_no_multichoice, cutoff=10000 yield columns=7, ACC=0.8133402275077559, in duration=24.267s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical_no_multichoice, cutoff=5000 yield columns=8, ACC=0.8138572905894519, in duration=21.668s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical_no_multichoice, cutoff=2500 yield columns=20, ACC=0.8143743536711479, in duration=21.956s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical_no_multichoice, cutoff=1000 yield columns=87, ACC=0.8200620475698035, in duration=33.154s
rowset_label=full_country_United States, columnset_label=all_numeric_categerical_no_multichoice, cutoff=500 yield columns=119, ACC=0.8226473629782833, in duration=48.716s
rowset_label=full_country_India, columnset_label=all_numeric_categerical_no_multichoice, cutoff=10000 yield columns=7, ACC=0.8640124095139607, in 

In [9]:
# result_df = pd.DataFrame(data=results).set_index(['rowset_label', 'columnset_label', 'country', 'cutoff'])
result_df = pd.DataFrame(data=results).set_index(['country', 'cutoff'])
for to_round in ['TPR', 'TNR', 'PPV', 'NPV', 'ACC', 'duration']:
    result_df[to_round] = result_df[to_round].map(lambda x: round(x,3))

result_df.drop(columns=['X_columns', 'coefficients']).sort_values('ACC')

TP  FP    TN   FN    TPR    TNR    PPV    NPV    ACC  \
country        cutoff                                                         
United States  10000   13  13  1560  348  0.036  0.992  0.500  0.818  0.813   
               5000    13  12  1561  348  0.036  0.992  0.520  0.818  0.814   
               2500    18  16  1557  343  0.050  0.990  0.529  0.819  0.814   
               1000    33  20  1553  328  0.091  0.987  0.623  0.826  0.820   
               500     42  24  1549  319  0.116  0.985  0.636  0.829  0.823   
India          10000    0   0  1671  263  0.000  1.000  0.000  0.864  0.864   
               5000     0   0  1671  263  0.000  1.000  0.000  0.864  0.864   
               2500     0   0  1671  263  0.000  1.000  0.000  0.864  0.864   
               1000    13   3  1668  250  0.049  0.998  0.812  0.870  0.869   
               500     18   9  1662  245  0.068  0.995  0.667  0.872  0.869   
Germany        500      0   0  1821  113  0.000  1.000  0.000  0.942  0.942   
               2500     0   0  1821  113  0.000  1.000  0.000  0.942  0.942   
               1000     0   0  1821  113  0.000  1.000  0.000  0.942  0.942   
               10000    0   0  1821  113  0.000  1.000  0.000  0.942  0.942   
               5000     0   0  1821  113  0.000  1.000  0.000  0.942  0.942   
United Kingdom 10000    0   0  1823  111  0.000  1.000  0.000  0.943  0.943   
               5000     0   0  1823  111  0.000  1.000  0.000  0.943  0.943   
               2500     0   0  1823  111  0.000  1.000  0.000  0.943  0.943   
               1000     0   0  1823  111  0.000  1.000  0.000  0.943  0.943   
               500      0   0  1823  111  0.000  1.000  0.000  0.943  0.943   
Brazil         10000    0   0  1874   60  0.000  1.000  0.000  0.969  0.969   
               5000     0   0  1874   60  0.000  1.000  0.000  0.969  0.969   
               2500     0   0  1874   60  0.000  1.000  0.000  0.969  0.969   
               1000     0   0  1874   60  0.000  1.000  0.000  0.969  0.969   
               500      0   0  1874   60  0.000  1.000  0.000  0.969  0.969   
Canada         500      0   0  1876   58  0.000  1.000  0.000  0.970  0.970   
               1000     0   0  1876   58  0.000  1.000  0.000  0.970  0.970   
               2500     0   0  1876   58  0.000  1.000  0.000  0.970  0.970   
               5000     0   0  1876   58  0.000  1.000  0.000  0.970  0.970   
               10000    0   0  1876   58  0.000  1.000  0.000  0.970  0.970   
Netherlands    500      0   0  1886   48  0.000  1.000  0.000  0.975  0.975   
               1000     0   0  1886   48  0.000  1.000  0.000  0.975  0.975   
               2500     0   0  1886   48  0.000  1.000  0.000  0.975  0.975   
               10000    0   0  1886   48  0.000  1.000  0.000  0.975  0.975   
               5000     0   0  1886   48  0.000  1.000  0.000  0.975  0.975   
France         1000     0   0  1887   47  0.000  1.000  0.000  0.976  0.976   
Australia      5000     0   0  1888   46  0.000  1.000  0.000  0.976  0.976   
France         2500     0   0  1887   47  0.000  1.000  0.000  0.976  0.976   
               5000     0   0  1887   47  0.000  1.000  0.000  0.976  0.976   
               10000    0   0  1887   47  0.000  1.000  0.000  0.976  0.976   
Australia      500      0   0  1888   46  0.000  1.000  0.000  0.976  0.976   
               1000     0   0  1888   46  0.000  1.000  0.000  0.976  0.976   
               2500     0   0  1888   46  0.000  1.000  0.000  0.976  0.976   
France         500      0   0  1887   47  0.000  1.000  0.000  0.976  0.976   
Australia      10000    0   0  1888   46  0.000  1.000  0.000  0.976  0.976   
Poland         5000     0   0  1896   38  0.000  1.000  0.000  0.980  0.980   
               1000     0   0  1896   38  0.000  1.000  0.000  0.980  0.980   
               2500     0   0  1896   38  0.000  1.000  0.000  0.980  0.980   
               10000    0   0  1896   38  0.000  1.000  0.000  0.980  0.9

In [7]:
script_end = time.time()
print('script duration %s', end - start)

script duration %s 5.375078916549683
